## Practica - Usos de la IA

Alumno: Hannibal Tomasson Izquierdo

### Objetivo:
Automatizar el procesamiento de solicitudes de devolucion recibidas por correo electronico en la empresa ficticia Componentes Intergalacticos Industriales S.A.. El sistema analiza el contenido del mensaje, determina si la solicitud debe ser aceptada o rechazada, y genera una respuesta formal personalizada con un codigo unico de seguimiento.


In [ ]:
# Instalamos las librerias necesarias
!pip install --quiet langchain langchain-openai fpdf

In [ ]:
# Hacemos los imports que vamos a usar
import os
import uuid
from datetime import datetime
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from fpdf import FPDF
from google.colab import files

In [ ]:
# Configuramos la clave de API de OpenAI
os.environ["OPENAI_API_KEY"] = "OPENAI_API_KEY" # He borrado la API key para poder subir el notebook a GitHub

# Inicializamos el modelo
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")

Primero de todo vamos a definir el email original recibido como el ejemplo que nuestro modelo va a usar para aprender la informacion que necesitamos que extraiga.

In [ ]:
email_original = """
Asunto: Solicitud de reemplazo por daños en transporte – Pedido #D347-STELLA

Estimado equipo de Componentes Intergalácticos Industriales S.A.,

Me pongo en contacto con ustedes como cliente reciente para comunicar una
incidencia relacionada con el pedido #D347-STELLA, correspondiente a un lote de
condensadores de fluzo modelo FX-88, destinados a un proyecto estratégico de gran
envergadura: la construcción de la Estrella de la Muerte.

Lamentablemente, al recibir el envío, observamos que varios de los condensadores
presentaban daños visibles y no funcionales. Tras revisar el estado del embalaje y
consultar con el piloto de carga, todo indica que la mercancía sufrió una caída
durante el transporte interestelar.

Dado que estos componentes son críticos para la activación del núcleo central del
sistema de rayos destructores, les solicitamos con carácter urgente el reemplazo
inmediato de las unidades defectuosas, así como una revisión de los protocolos de
embalaje y transporte para evitar que algo así vuelva a ocurrir.

Adjunto imágenes del estado de los condensadores y el albarán de entrega sellado
por nuestro droide de recepción.

Agradezco de antemano su pronta atención a este asunto. Quedamos a la espera de
su respuesta para coordinar el reemplazo.

Atentamente,
Darth Márquez
Departamento de Ingeniería Imperial
Sector de Proyectos Especiales
Contacto: dmarquez@imperiumgalactic.net
Holofono: +34 9X9 123 456
"""


Luego vamos a definir un identificador unico para cada email que incluya la fecha actual con un codigo especifico creado ad hoc.

In [ ]:
fecha_hoy = datetime.now().strftime("%Y%m%d")
codigo_unico = f"CII-R-{fecha_hoy}-{uuid.uuid4().hex[:4].upper()}"
email_con_codigo = f"[Código de seguimiento: {codigo_unico}]\n\n{email_original}"

Ahora creamos la plantilla que especifica la informacion que queremos extraer de los correos que recibamos.

In [ ]:
# Plantilla para extraer la informacion del correo
extraction_prompt = PromptTemplate(
    input_variables=["email"],
    template="""
A continuación tienes un email recibido por la empresa "Componentes Intergalácticos Industriales S.A.".

Extrae del texto los siguientes datos y responde en formato JSON:

- Nombre del remitente
- Correo electronico
- Numero de pedido
- Motivo de la devoluicion (resumido en una frase)
- ¿El cliente menciona daños visibles? (si/no)
- ¿El cliente menciona si el daño ocurrio durante el transporte? (si/no)
- ¿Se adjunta algun archivo o evidencia? (si/no)
- ¿Se solicita un reemplazo? (si/no)

Email:
---------------------
{email}
---------------------
JSON:
"""
)

extractor_chain = LLMChain(llm=llm, prompt=extraction_prompt)

In [ ]:
extracted_info = extractor_chain.invoke({"email": email_con_codigo})["text"]
print(extracted_info)

{
    "Nombre del remitente": "Darth Márquez",
    "Correo electronico": "dmarquez@imperiumgalactic.net",
    "Numero de pedido": "#D347-STELLA",
    "Motivo de la devoluicion": "Daños en transporte de condensadores de fluzo modelo FX-88",
    "¿El cliente menciona daños visibles?": "si",
    "¿El cliente menciona si el daño ocurrio durante el transporte?": "si",
    "¿Se adjunta algun archivo o evidencia?": "si",
    "¿Se solicita un reemplazo?": "si"
}


Lo siguiente que haremos sera crear el prompt que va a tomar una decision sobre si la devolucion se debe aceptar o declinar, y que la tomara en base a la informacion extraida en el paso anterior.

In [ ]:
decision_prompt = PromptTemplate(
    input_variables=["extracted_info"],
    template="""
A continuacion tienes la informacion extraida de un email de solicitud de devolucion:

{extracted_info}

Debes decidir si la devolucion debe ser ACEPTADA o RECHAZADA segun las siguientes reglas:

ACEPTAR si:
- Hay un defecto de fabricacion confirmado.
- Se entregaron componentes incorrectos (modelo, cantidad o especificacion).
- El producto esta incompleto desde fabrica.

RECHAZAR si:
- El daño ocurrio durante el transporte.
- Hubo manipulacion indebida por parte del cliente.
- Se supero el plazo maximo de devolucion (14 dias naturales).


Si el daño ocurrió durante el transporte y no se especifica que la empresa fue responsable del mismo, la devolución debe RECHAZARSE por defecto.
Indica claramente si la devolucion debe ser ACEPTADA o RECHAZADA. Explica brevemente por que, y termina con la palabra clave: "DECISION: ACEPTADA" o "DECISION: RECHAZADA".
"""
)

decision_chain = LLMChain(llm=llm, prompt=decision_prompt)

In [ ]:
decision_result = decision_chain.invoke({"extracted_info": extracted_info})["text"]
print(decision_result)


La devolución debe ser RECHAZADA. El cliente menciona que los daños ocurrieron durante el transporte, por lo tanto, no se puede confirmar un defecto de fabricación. Además, no se especifica que la empresa fue responsable del daño en el transporte. Por lo tanto, según las reglas establecidas, la devolución debe ser rechazada. 

DECISION: RECHAZADA


El siguiente paso es crear el prompt que elabore una respuesta adecuada al mensaje que hemos recibido. En este prompt definiremos como debe responder el sistema, y posteriormente guardaremos todo el intercambio de informacion con el codigo que se le habia ya asignado.

In [ ]:
response_prompt = PromptTemplate(
    input_variables=["extracted_info", "decision_text", "codigo_seguimiento"],
    template="""
Eres un asistente profesional de atencion al cliente en la empresa "Componentes Intergalacticos Industriales S.A."

Tienes los siguientes datos extraidos del email del cliente:

{extracted_info}

Y esta es la decision sobre la devolucion:

{decision_text}

Redacta un correo de respuesta formal y profesional, pero empatico. Incluye:
- Un saludo personalizado con el nombre del remitente
- Un agradecimiento por contactarnos
- Una mencion clara sobre la decision tomada (aceptada o rechazada)
- Una explicacion breve de la razón
- Siguientes pasos (por ejemplo: confirmacion de reemplazo, envio del nuevo lote, o contacto con soporte si fue rechazada)
- Firma institucional

Escribe el mensaje en español.
"""
)

response_chain = LLMChain(llm=llm, prompt=response_prompt)

In [ ]:
final_response = response_chain.invoke({
    "extracted_info": extracted_info,
    "decision_text": decision_result,
    "codigo_seguimiento": codigo_unico
})["text"]

print(final_response)

Estimado Darth Márquez,

Agradecemos que te hayas puesto en contacto con nosotros para informarnos sobre los daños en el transporte de los condensadores de fluzo modelo FX-88 de tu pedido #D347-STELLA.

Lamentablemente, después de revisar detenidamente la situación, hemos determinado que la devolución debe ser RECHAZADA. Según lo mencionado, los daños ocurrieron durante el transporte y no se puede confirmar un defecto de fabricación. Además, no se especifica que la empresa fue responsable del daño en el transporte.

Si necesitas más asistencia o información adicional, por favor no dudes en contactar a nuestro equipo de soporte para que podamos ayudarte de la mejor manera posible.

Gracias por tu comprensión y paciencia.

Atentamente,

[Nombre]
Asistente de Atención al Cliente
Componentes Intergalácticos Industriales S.A.


In [ ]:
respuesta_final = f"Código de seguimiento: {codigo_unico}\n\n{final_response}"
filename = f"respuesta_{codigo_unico}.txt"
with open(filename, "w", encoding="utf-8") as f:
    f.write(respuesta_final)

files.download(filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Documento justificativo: Automatizacion con LangChain

### Estructura del flujo y justificacion:

Paso 1 – Generacion de codigo unico:
Cada correo recibido se asocia a un identificador irrepetible (CII-R-YYYYMMDD-XXXX), lo que permite trazar el caso de forma segura y organizada a lo largo del flujo. Este codigo se anade al inicio del correo y a la respuesta final.

Paso 2 – Extraccion de informacion relevante:
Se utiliza una LLMChain con un PromptTemplate que extrae automaticamente los datos clave del correo (nombre, motivo, numero de pedido, etc.). Este paso permite convertir un mensaje en lenguaje natural en datos estructurados utiles para la evaluacion.

Paso 3 – Evaluacion de condiciones de devolucion:
Otra LLMChain evalua los criterios definidos por la politica de la empresa. En este paso, se introdujo una mejora clave:

Si se detecta que el producto sufrio danos durante el transporte y no se especifica claramente que la empresa fue responsable del envio, la devolucion se rechaza automaticamente por defecto.
Esta modificacion evita que el modelo acepte solicitudes por error debido a ambiguedades comunes en correos reales.

Paso 4 – Generacion de respuesta automatica personalizada:
Con la informacion extraida y la decision tomada, una tercera cadena genera un correo formal y empatico que incluye saludo, resolucion, pasos a seguir y la firma institucional con el codigo de seguimiento.

Paso 5 – Guardado y descarga automatica:
La respuesta final se guarda en un archivo .txt con el nombre del codigo de seguimiento, lo cual permite archivar facilmente los casos y/o enviar el archivo directamente al cliente.

### Justificacion tecnica:
El uso de LangChain permite dividir el flujo en etapas claras (extraccion, decision, respuesta), cada una basada en prompts bien definidos. Esto mejora la mantenibilidad, la trazabilidad y la transparencia del sistema.
La inclusion de una politica de rechazo por defecto ante la ambigüedad refuerza la solidez del sistema y reduce el margen de error al automatizar decisiones criticas.